In [1]:
# cnn lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot

Using TensorFlow backend.


In [2]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

In [3]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

In [4]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [5]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [7]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    # define model
    verbose, epochs, batch_size = 1, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # reshape data into time steps of sub-sequences
    n_steps, n_length = 4, 32
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    # define model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=trainX.shape[1:]))#(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    from keras.callbacks import ModelCheckpoint, EarlyStopping
    checkpoint = ModelCheckpoint("CNN_LSTM_Test_2.h5", monitor='accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    early = EarlyStopping(monitor='accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

    
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks=[checkpoint,early])
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
    return accuracy

In [8]:
# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [9]:
# run an experiment
def run_experiment(repeats=10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [10]:
# run the experiment
run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
Epoch 1/25
7352/7352 [==============================] - 5s 619us/step - loss: 0.5895 - accuracy: 0.7571

Epoch 00001: accuracy improved from -inf to 0.75707, saving model to CNN_LSTM_Test_2.h5
Epoch 2/25
7352/7352 [==============================] - 4s 572us/step - loss: 0.2038 - accuracy: 0.9207

Epoch 00002: accuracy improved from 0.75707 to 0.92070, saving model to CNN_LSTM_Test_2.h5
Epoch 3/25
7352/7352 [==============================] - 4s 555us/step - loss: 0.1540 - accuracy: 0.9408

Epoch 00003: accuracy improved from 0.92070 to 0.94083, saving model to CNN_LSTM_Test_2.h5
Epoch 4/25
7352/7352 [==============================] - 4s 558us/step - loss: 0.1275 - accuracy: 0.9465

Epoch 00004: accuracy improved from 0.94083 to 0.94655, saving model to CNN_LSTM_Test_2.h5
Epoch 5/25
7352/7352 [==============================] - 4s 558us/step - loss: 0.1160 - accuracy: 0.9504

Epoch 00005: a

Epoch 21/25
7352/7352 [==============================] - 5s 720us/step - loss: 0.0708 - accuracy: 0.9633

Epoch 00021: accuracy did not improve from 0.96518
Epoch 22/25
7352/7352 [==============================] - 5s 690us/step - loss: 0.0769 - accuracy: 0.9627

Epoch 00022: accuracy did not improve from 0.96518
Epoch 23/25
7352/7352 [==============================] - 5s 685us/step - loss: 0.0724 - accuracy: 0.9615

Epoch 00023: accuracy did not improve from 0.96518
Epoch 24/25
7352/7352 [==============================] - 5s 715us/step - loss: 0.0659 - accuracy: 0.9668

Epoch 00024: accuracy improved from 0.96518 to 0.96681, saving model to CNN_LSTM_Test_2.h5
Epoch 25/25
7352/7352 [==============================] - 5s 682us/step - loss: 0.0743 - accuracy: 0.9635

Epoch 00025: accuracy did not improve from 0.96681
2947/2947 [==============================] - 0s 164us/step
>#2: 92.535
Epoch 1/25
7352/7352 [==============================] - 5s 721us/step - loss: 0.6260 - accuracy: 0.7418


Epoch 16/25
7352/7352 [==============================] - 5s 712us/step - loss: 0.0896 - accuracy: 0.95590s - loss: 0.0884 - accuracy

Epoch 00016: accuracy did not improve from 0.95688
Epoch 17/25
7352/7352 [==============================] - 5s 716us/step - loss: 0.0970 - accuracy: 0.9573

Epoch 00017: accuracy improved from 0.95688 to 0.95729, saving model to CNN_LSTM_Test_2.h5
Epoch 18/25
7352/7352 [==============================] - 5s 746us/step - loss: 0.0964 - accuracy: 0.9554

Epoch 00018: accuracy did not improve from 0.95729
Epoch 19/25
7352/7352 [==============================] - 5s 744us/step - loss: 0.0870 - accuracy: 0.9576

Epoch 00019: accuracy improved from 0.95729 to 0.95756, saving model to CNN_LSTM_Test_2.h5
Epoch 20/25
7352/7352 [==============================] - 5s 708us/step - loss: 0.0885 - accuracy: 0.9563

Epoch 00020: accuracy did not improve from 0.95756
Epoch 21/25
7352/7352 [==============================] - 5s 722us/step - loss: 0.0875 - accuracy: 0.9603

E

7352/7352 [==============================] - 5s 741us/step - loss: 0.1113 - accuracy: 0.9514

Epoch 00010: accuracy did not improve from 0.95770
Epoch 11/25
7352/7352 [==============================] - 6s 768us/step - loss: 0.1034 - accuracy: 0.9544

Epoch 00011: accuracy did not improve from 0.95770
Epoch 12/25
7352/7352 [==============================] - 6s 765us/step - loss: 0.1072 - accuracy: 0.9550

Epoch 00012: accuracy did not improve from 0.95770
Epoch 13/25
7352/7352 [==============================] - 6s 862us/step - loss: 0.0978 - accuracy: 0.9577

Epoch 00013: accuracy did not improve from 0.95770
Epoch 14/25
7352/7352 [==============================] - 7s 908us/step - loss: 0.1062 - accuracy: 0.9547

Epoch 00014: accuracy did not improve from 0.95770
Epoch 15/25
7352/7352 [==============================] - 6s 780us/step - loss: 0.1035 - accuracy: 0.9529

Epoch 00015: accuracy did not improve from 0.95770
Epoch 16/25
7352/7352 [==============================] - 5s 685us/step

7352/7352 [==============================] - 5s 685us/step - loss: 0.1231 - accuracy: 0.9505

Epoch 00005: accuracy improved from 0.94478 to 0.95049, saving model to CNN_LSTM_Test_2.h5
Epoch 6/25
7352/7352 [==============================] - 5s 742us/step - loss: 0.1110 - accuracy: 0.9555

Epoch 00006: accuracy improved from 0.95049 to 0.95552, saving model to CNN_LSTM_Test_2.h5
Epoch 7/25
7352/7352 [==============================] - 5s 705us/step - loss: 0.1143 - accuracy: 0.9509

Epoch 00007: accuracy did not improve from 0.95552
Epoch 8/25
7352/7352 [==============================] - 5s 673us/step - loss: 0.1029 - accuracy: 0.9551

Epoch 00008: accuracy did not improve from 0.95552
Epoch 9/25
7352/7352 [==============================] - 5s 705us/step - loss: 0.0924 - accuracy: 0.95990s - loss: 0.0

Epoch 00009: accuracy improved from 0.95552 to 0.95987, saving model to CNN_LSTM_Test_2.h5
Epoch 10/25
7352/7352 [==============================] - 5s 683us/step - loss: 0.0890 - accuracy:

2947/2947 [==============================] - 1s 173us/step
>#9: 91.042
Epoch 1/25
7352/7352 [==============================] - 5s 724us/step - loss: 0.6606 - accuracy: 0.7267

Epoch 00001: accuracy improved from -inf to 0.72674, saving model to CNN_LSTM_Test_2.h5
Epoch 2/25
7352/7352 [==============================] - 5s 667us/step - loss: 0.2270 - accuracy: 0.9151

Epoch 00002: accuracy improved from 0.72674 to 0.91513, saving model to CNN_LSTM_Test_2.h5
Epoch 3/25
7352/7352 [==============================] - 5s 664us/step - loss: 0.1496 - accuracy: 0.9406

Epoch 00003: accuracy improved from 0.91513 to 0.94056, saving model to CNN_LSTM_Test_2.h5
Epoch 4/25
7352/7352 [==============================] - 5s 694us/step - loss: 0.1256 - accuracy: 0.9498

Epoch 00004: accuracy improved from 0.94056 to 0.94981, saving model to CNN_LSTM_Test_2.h5
Epoch 5/25
7352/7352 [==============================] - 5s 668us/step - loss: 0.1247 - accuracy: 0.9486

Epoch 00005: accuracy did not improve from 